In [1]:
import catboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('dataset/prep.csv')

# Preprocessing

### Change data to use CatBoost

In [3]:
def replace_from_dict(dct, column):
    return list(map(lambda x: dct.get(x) if x in dct else x, data[column]))

#### Cuzov

In [4]:
dict_cuzov = {'седан':'sedan', 
               'хэтчбек':'hatchback',
               'внедорожник':'SUV',
               'универсал':'universal',
               'минивэн':'minivan',
               'лифтбек':'liftback',
               'микроавтобус':'minibus',
               'купе':'coupe',
               'легковой':'passenger',
               'пикап':'pickup',
               'кабриолет':'cabriolet',
               'другой':'other',
               'лимузин':'limousine',
               'родстер':'roadster'               
}

In [5]:
data.cuzov = replace_from_dict(dict_cuzov, 'cuzov')

#### Fuel

In [6]:
dict_fuel = {'бензин':'petrol', 
             'дизель':'diesel'               
}

In [7]:
data.fuel = replace_from_dict(dict_fuel, 'fuel')

#### Brand

In [8]:
dict_brand = {'ГАЗ':'GAZ', 
             'УАЗ':'UAZ',
             'Москвич':'Moskvich', 
             'ЗАЗ':'ZAZ', 
             'Эксклюзив':'Exclusive',
             'ЛуАЗ':'LuAZ'
}

In [9]:
data.brand = replace_from_dict(dict_brand, 'brand')

#### Region

In [10]:
dict_region = {'Минск':'Minsk', 
               'Брестская обл.':'Brest reg.',
               'Гродненская обл.':'Grodno reg.', 
               'Гомельская обл.':'Gomel reg.', 
               'Могилевская обл.':'Mogilev reg.',
               'Витебская обл.':'Vitebsk reg.',
               'Минская обл.':'Minsk reg.'
}

In [11]:
data.region = replace_from_dict(dict_region, 'region')

#### Model - drop

In [12]:
data.drop('model', axis=1, inplace=True)

#### Color

In [13]:
data.color.unique()

array(['черный', 'серебристый', 'зеленый', 'фиолетовый', 'синий',
       'бордовый', 'серый', 'белый', 'другой', 'красный', 'коричневый',
       'оранжевый', 'желтый'], dtype=object)

In [14]:
russian_colors = ['черный', 'серебристый', 'зеленый', 'фиолетовый', 'синий',
                  'бордовый', 'серый', 'белый', 'другой', 'красный', 
                  'коричневый', 'оранжевый', 'желтый']
english_colors = ['black', 'silver', 'green', 'purple', 'blue',
                  'burgundy', 'gray', 'white', 'other', 'red',
                  'brown', 'orange', 'yellow']
dict_colors = dict(zip(russian_colors, english_colors))

In [15]:
data.color = replace_from_dict(dict_colors, 'color')

#### Drive Unit

In [16]:
dict_drive_unit = {'Передний':'front-wheel', 
                   'Постоянный полный':'all-wheel', 
                   'Задний':'rear', 
                   'Подключаемый полный':'four-wheel'               
}

In [17]:
data['drive-unit'] = replace_from_dict(dict_drive_unit, 'drive-unit')

In [18]:
data['drive-unit'].fillna('Unknown', inplace=True)

#### State

In [19]:
dict_state = {'с пробегом':'with mileage', 
              'с повреждениями':'with damage', 
              'на запчасти':'for parts'               
}

In [20]:
data.state = replace_from_dict(dict_state, 'state')

#### Transmission

In [21]:
dict_transmission = {'Автомат':'automatic', 
                     'Механика':'mechanical'               
}

In [22]:
data.transmission = replace_from_dict(dict_transmission, 'transmission')

#### Drop views

In [23]:
data.drop(['all_views', 'today_views', 'up', 'days_ago'], axis=1, inplace=True)

In [24]:
data.drop('modified', axis=1, inplace=True)

In [25]:
data.cost = data.cost.str.replace(' ', '').astype('int32')

# Train Test Split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('cost', axis=1), 
                                                    data.cost, random_state=42,
                                                    test_size=0.2)

# Remove Outliers

In [40]:
count_brand = X_train.brand.value_counts()
count_cuzov = X_train.cuzov.value_counts()

In [41]:
data = data[(list(count_brand[data.brand]>15))]
data = data[list(count_cuzov[data.cuzov]>20)]

# Try CV

In [42]:
ctbst = catboost.CatBoostRegressor(cat_features=['cuzov', 'fuel', 'brand', 'region',
                                                 'color', 'drive-unit', 'state', 'transmission'],
                                   od_pval=1e-5)

In [45]:
score_cv = cross_val_score(ctbst, X_train, y=y_train, cv=5)

0:	learn: 9465.7172674	total: 41.8ms	remaining: 41.7s
1:	learn: 9270.3840557	total: 86.9ms	remaining: 43.3s
2:	learn: 9080.9778521	total: 133ms	remaining: 44.2s
3:	learn: 8917.4576285	total: 179ms	remaining: 44.6s
4:	learn: 8744.7678519	total: 227ms	remaining: 45.1s
5:	learn: 8569.9147266	total: 283ms	remaining: 46.8s
6:	learn: 8409.6681447	total: 331ms	remaining: 46.9s
7:	learn: 8240.4133885	total: 379ms	remaining: 47s
8:	learn: 8091.1454336	total: 427ms	remaining: 47s
9:	learn: 7943.8337617	total: 475ms	remaining: 47s
10:	learn: 7799.0328480	total: 526ms	remaining: 47.3s
11:	learn: 7660.4866399	total: 571ms	remaining: 47s
12:	learn: 7532.0151143	total: 605ms	remaining: 45.9s
13:	learn: 7410.1243211	total: 653ms	remaining: 46s
14:	learn: 7292.9895607	total: 699ms	remaining: 45.9s
15:	learn: 7165.4986829	total: 751ms	remaining: 46.2s
16:	learn: 7042.6310855	total: 799ms	remaining: 46.2s
17:	learn: 6923.7497469	total: 844ms	remaining: 46.1s
18:	learn: 6815.4523245	total: 892ms	remaining

154:	learn: 3333.6663794	total: 7.96s	remaining: 43.4s
155:	learn: 3328.8856032	total: 8.01s	remaining: 43.3s
156:	learn: 3321.9950460	total: 8.05s	remaining: 43.2s
157:	learn: 3313.7537003	total: 8.1s	remaining: 43.2s
158:	learn: 3308.7132100	total: 8.14s	remaining: 43.1s
159:	learn: 3304.9465687	total: 8.2s	remaining: 43.1s
160:	learn: 3300.2501056	total: 8.25s	remaining: 43s
161:	learn: 3293.7016167	total: 8.3s	remaining: 42.9s
162:	learn: 3288.0749553	total: 8.35s	remaining: 42.9s
163:	learn: 3281.5959585	total: 8.4s	remaining: 42.8s
164:	learn: 3270.7117170	total: 8.45s	remaining: 42.8s
165:	learn: 3268.3008313	total: 8.5s	remaining: 42.7s
166:	learn: 3265.5773884	total: 8.55s	remaining: 42.6s
167:	learn: 3263.4761144	total: 8.59s	remaining: 42.6s
168:	learn: 3255.7350723	total: 8.64s	remaining: 42.5s
169:	learn: 3252.3542600	total: 8.7s	remaining: 42.5s
170:	learn: 3250.3478597	total: 8.75s	remaining: 42.4s
171:	learn: 3245.9055440	total: 8.79s	remaining: 42.3s
172:	learn: 3242.4

307:	learn: 2983.1044023	total: 16s	remaining: 36s
308:	learn: 2983.0747442	total: 16.1s	remaining: 35.9s
309:	learn: 2982.5153247	total: 16.1s	remaining: 35.9s
310:	learn: 2981.5776972	total: 16.2s	remaining: 35.8s
311:	learn: 2981.1688205	total: 16.2s	remaining: 35.8s
312:	learn: 2981.0693426	total: 16.3s	remaining: 35.7s
313:	learn: 2980.5423840	total: 16.3s	remaining: 35.7s
314:	learn: 2979.9927574	total: 16.4s	remaining: 35.6s
315:	learn: 2978.7748050	total: 16.4s	remaining: 35.5s
316:	learn: 2978.3747756	total: 16.5s	remaining: 35.5s
317:	learn: 2977.7655390	total: 16.5s	remaining: 35.4s
318:	learn: 2977.1050106	total: 16.6s	remaining: 35.4s
319:	learn: 2976.7000696	total: 16.6s	remaining: 35.3s
320:	learn: 2976.4100773	total: 16.7s	remaining: 35.3s
321:	learn: 2975.6282145	total: 16.7s	remaining: 35.2s
322:	learn: 2974.5608806	total: 16.8s	remaining: 35.1s
323:	learn: 2974.0179140	total: 16.8s	remaining: 35.1s
324:	learn: 2973.9707120	total: 16.9s	remaining: 35s
325:	learn: 2973

458:	learn: 2964.5456260	total: 21.9s	remaining: 25.8s
459:	learn: 2964.5453345	total: 21.9s	remaining: 25.7s
460:	learn: 2964.5379566	total: 21.9s	remaining: 25.6s
461:	learn: 2964.5376559	total: 21.9s	remaining: 25.5s
462:	learn: 2964.5329394	total: 22s	remaining: 25.5s
463:	learn: 2964.5300868	total: 22s	remaining: 25.4s
464:	learn: 2964.5295706	total: 22s	remaining: 25.3s
465:	learn: 2964.5243266	total: 22.1s	remaining: 25.3s
466:	learn: 2964.5231827	total: 22.1s	remaining: 25.2s
467:	learn: 2964.5203282	total: 22.1s	remaining: 25.2s
468:	learn: 2964.5093721	total: 22.2s	remaining: 25.1s
469:	learn: 2964.5058781	total: 22.2s	remaining: 25s
470:	learn: 2964.5031945	total: 22.2s	remaining: 25s
471:	learn: 2964.5005908	total: 22.3s	remaining: 24.9s
472:	learn: 2964.5004039	total: 22.3s	remaining: 24.8s
473:	learn: 2964.5002235	total: 22.3s	remaining: 24.8s
474:	learn: 2964.4935837	total: 22.4s	remaining: 24.7s
475:	learn: 2964.4895010	total: 22.4s	remaining: 24.6s
476:	learn: 2964.486

611:	learn: 2964.0388801	total: 25.7s	remaining: 16.3s
612:	learn: 2964.0388462	total: 25.8s	remaining: 16.3s
613:	learn: 2964.0379705	total: 25.8s	remaining: 16.2s
614:	learn: 2964.0371209	total: 25.8s	remaining: 16.1s
615:	learn: 2964.0362966	total: 25.8s	remaining: 16.1s
616:	learn: 2964.0354968	total: 25.8s	remaining: 16s
617:	learn: 2964.0347209	total: 25.8s	remaining: 16s
618:	learn: 2964.0339680	total: 25.8s	remaining: 15.9s
619:	learn: 2964.0332376	total: 25.9s	remaining: 15.8s
620:	learn: 2964.0325290	total: 25.9s	remaining: 15.8s
621:	learn: 2964.0325289	total: 25.9s	remaining: 15.7s
622:	learn: 2964.0324960	total: 25.9s	remaining: 15.7s
623:	learn: 2964.0324960	total: 25.9s	remaining: 15.6s
624:	learn: 2964.0318084	total: 25.9s	remaining: 15.6s
625:	learn: 2964.0311413	total: 25.9s	remaining: 15.5s
626:	learn: 2964.0311091	total: 26s	remaining: 15.4s
627:	learn: 2964.0310776	total: 26s	remaining: 15.4s
628:	learn: 2964.0304302	total: 26s	remaining: 15.3s
629:	learn: 2964.028

...

284:	learn: 3017.0912432	total: 14.7s	remaining: 37s
285:	learn: 3016.5586461	total: 14.8s	remaining: 36.9s
286:	learn: 3016.5164380	total: 14.8s	remaining: 36.8s
287:	learn: 3014.6618995	total: 14.9s	remaining: 36.8s
288:	learn: 3013.5756613	total: 14.9s	remaining: 36.7s
289:	learn: 3013.3912707	total: 15s	remaining: 36.7s
290:	learn: 3012.3570543	total: 15s	remaining: 36.6s
291:	learn: 3011.0484674	total: 15.1s	remaining: 36.5s
292:	learn: 3009.7768734	total: 15.1s	remaining: 36.5s
293:	learn: 3009.7342670	total: 15.2s	remaining: 36.4s
294:	learn: 3008.6332952	total: 15.2s	remaining: 36.4s
295:	learn: 3007.9908529	total: 15.3s	remaining: 36.3s
296:	learn: 3006.8024938	total: 15.3s	remaining: 36.3s
297:	learn: 3005.0628025	total: 15.4s	remaining: 36.2s
298:	learn: 3003.4776750	total: 15.4s	remaining: 36.1s
299:	learn: 3003.2028729	total: 15.5s	remaining: 36.1s
300:	learn: 3002.7821858	total: 15.5s	remaining: 36s
301:	learn: 3002.7390108	total: 15.6s	remaining: 36s
302:	learn: 3001.458

438:	learn: 2970.8136515	total: 22.1s	remaining: 28.2s
439:	learn: 2970.8126508	total: 22.1s	remaining: 28.1s
440:	learn: 2970.7883761	total: 22.2s	remaining: 28.1s
441:	learn: 2970.7797841	total: 22.2s	remaining: 28s
442:	learn: 2970.7563789	total: 22.3s	remaining: 28s
443:	learn: 2970.5229229	total: 22.3s	remaining: 28s
444:	learn: 2970.2238168	total: 22.4s	remaining: 27.9s
445:	learn: 2970.1263464	total: 22.5s	remaining: 27.9s
446:	learn: 2968.5362127	total: 22.5s	remaining: 27.9s
447:	learn: 2968.5352833	total: 22.6s	remaining: 27.8s
448:	learn: 2967.5870719	total: 22.6s	remaining: 27.8s
449:	learn: 2967.3045549	total: 22.7s	remaining: 27.7s
450:	learn: 2967.0928760	total: 22.7s	remaining: 27.7s
451:	learn: 2967.0821701	total: 22.8s	remaining: 27.6s
452:	learn: 2967.0716831	total: 22.8s	remaining: 27.6s
453:	learn: 2966.3518257	total: 22.9s	remaining: 27.5s
454:	learn: 2965.8722585	total: 23s	remaining: 27.5s
455:	learn: 2965.8491636	total: 23s	remaining: 27.5s
456:	learn: 2965.848

593:	learn: 2953.5365605	total: 29.8s	remaining: 20.4s
594:	learn: 2953.5265949	total: 29.8s	remaining: 20.3s
595:	learn: 2953.5206461	total: 29.9s	remaining: 20.2s
596:	learn: 2953.5148601	total: 29.9s	remaining: 20.2s
597:	learn: 2953.5092311	total: 29.9s	remaining: 20.1s
598:	learn: 2953.5091227	total: 30s	remaining: 20.1s
599:	learn: 2953.5056710	total: 30s	remaining: 20s
600:	learn: 2953.4837454	total: 30.1s	remaining: 20s
601:	learn: 2953.4783193	total: 30.1s	remaining: 19.9s
602:	learn: 2953.4770341	total: 30.1s	remaining: 19.8s
603:	learn: 2953.4769336	total: 30.2s	remaining: 19.8s
604:	learn: 2953.4716527	total: 30.2s	remaining: 19.7s
605:	learn: 2953.4679499	total: 30.2s	remaining: 19.7s
606:	learn: 2953.4645626	total: 30.3s	remaining: 19.6s
607:	learn: 2953.4612419	total: 30.3s	remaining: 19.5s
608:	learn: 2953.4579567	total: 30.3s	remaining: 19.5s
609:	learn: 2953.4529618	total: 30.4s	remaining: 19.4s
610:	learn: 2953.4479779	total: 30.4s	remaining: 19.4s
611:	learn: 2953.4

748:	learn: 2953.0916948	total: 36s	remaining: 12.1s
749:	learn: 2953.0731779	total: 36s	remaining: 12s
750:	learn: 2953.0693315	total: 36.1s	remaining: 12s
751:	learn: 2953.0688627	total: 36.1s	remaining: 11.9s
752:	learn: 2953.0688429	total: 36.2s	remaining: 11.9s
753:	learn: 2953.0683931	total: 36.2s	remaining: 11.8s
754:	learn: 2953.0683741	total: 36.2s	remaining: 11.8s
755:	learn: 2953.0678885	total: 36.3s	remaining: 11.7s
756:	learn: 2953.0674114	total: 36.3s	remaining: 11.7s
757:	learn: 2953.0669814	total: 36.4s	remaining: 11.6s
758:	learn: 2953.0669634	total: 36.4s	remaining: 11.6s
759:	learn: 2953.0665403	total: 36.4s	remaining: 11.5s
760:	learn: 2953.0661234	total: 36.5s	remaining: 11.5s
761:	learn: 2953.0625235	total: 36.5s	remaining: 11.4s
762:	learn: 2953.0590879	total: 36.6s	remaining: 11.4s
763:	learn: 2953.0558092	total: 36.6s	remaining: 11.3s
764:	learn: 2953.0526802	total: 36.7s	remaining: 11.3s
765:	learn: 2953.0518769	total: 36.7s	remaining: 11.2s
766:	learn: 2953.0

901:	learn: 2952.9625142	total: 42.8s	remaining: 4.64s
902:	learn: 2952.9624487	total: 42.8s	remaining: 4.6s
903:	learn: 2952.9616618	total: 42.8s	remaining: 4.55s
904:	learn: 2952.9610538	total: 42.9s	remaining: 4.5s
905:	learn: 2952.9603029	total: 42.9s	remaining: 4.45s
906:	learn: 2952.9602456	total: 43s	remaining: 4.4s
907:	learn: 2952.9601856	total: 43s	remaining: 4.36s
908:	learn: 2952.9594722	total: 43s	remaining: 4.31s
909:	learn: 2952.9587914	total: 43.1s	remaining: 4.26s
910:	learn: 2952.9581417	total: 43.1s	remaining: 4.21s
911:	learn: 2952.9576970	total: 43.2s	remaining: 4.16s
912:	learn: 2952.9576427	total: 43.2s	remaining: 4.12s
913:	learn: 2952.9572075	total: 43.2s	remaining: 4.07s
914:	learn: 2952.9565890	total: 43.3s	remaining: 4.02s
915:	learn: 2952.9559992	total: 43.3s	remaining: 3.97s
916:	learn: 2952.9559472	total: 43.3s	remaining: 3.92s
917:	learn: 2952.9553583	total: 43.4s	remaining: 3.88s
918:	learn: 2952.9547963	total: 43.4s	remaining: 3.83s
919:	learn: 2952.95

In [48]:
print(f'Average R2-score in cv: {np.mean(score_cv):.6}')

Average R2-score in cv: 0.882778


# Try on Test Data

In [50]:
ctbst.fit(X_train, y_train)

0:	learn: 9407.9299563	total: 35.2ms	remaining: 35.1s
1:	learn: 9207.1475681	total: 82.2ms	remaining: 41s
2:	learn: 9022.3255722	total: 133ms	remaining: 44s
3:	learn: 8835.5163931	total: 193ms	remaining: 48.1s
4:	learn: 8659.9598067	total: 241ms	remaining: 48s
5:	learn: 8478.4061301	total: 292ms	remaining: 48.4s
6:	learn: 8317.0589582	total: 347ms	remaining: 49.2s
7:	learn: 8161.4510519	total: 393ms	remaining: 48.7s
8:	learn: 8008.2510563	total: 442ms	remaining: 48.7s
9:	learn: 7865.1245882	total: 495ms	remaining: 49s
10:	learn: 7719.5011235	total: 542ms	remaining: 48.8s
11:	learn: 7577.6829866	total: 588ms	remaining: 48.4s
12:	learn: 7443.9160593	total: 635ms	remaining: 48.2s
13:	learn: 7316.5157298	total: 687ms	remaining: 48.4s
14:	learn: 7184.6283383	total: 735ms	remaining: 48.2s
15:	learn: 7054.3641295	total: 793ms	remaining: 48.8s
16:	learn: 6948.0125641	total: 842ms	remaining: 48.7s
17:	learn: 6828.5765455	total: 891ms	remaining: 48.6s
18:	learn: 6718.1774470	total: 947ms	remaini

156:	learn: 3235.5461160	total: 8.04s	remaining: 43.2s
157:	learn: 3229.6245566	total: 8.09s	remaining: 43.1s
158:	learn: 3225.6943510	total: 8.14s	remaining: 43.1s
159:	learn: 3221.1297672	total: 8.19s	remaining: 43s
160:	learn: 3214.3919579	total: 8.24s	remaining: 43s
161:	learn: 3212.1763048	total: 8.3s	remaining: 42.9s
162:	learn: 3209.4178958	total: 8.35s	remaining: 42.9s
163:	learn: 3207.9152721	total: 8.4s	remaining: 42.8s
164:	learn: 3203.2541764	total: 8.45s	remaining: 42.8s
165:	learn: 3199.0787307	total: 8.51s	remaining: 42.8s
166:	learn: 3196.5275832	total: 8.58s	remaining: 42.8s
167:	learn: 3192.2215584	total: 8.64s	remaining: 42.8s
168:	learn: 3187.8350477	total: 8.7s	remaining: 42.8s
169:	learn: 3181.0025461	total: 8.76s	remaining: 42.8s
170:	learn: 3173.9488561	total: 8.82s	remaining: 42.8s
171:	learn: 3170.4303445	total: 8.88s	remaining: 42.8s
172:	learn: 3167.6975510	total: 8.94s	remaining: 42.8s
173:	learn: 3163.7692160	total: 9.01s	remaining: 42.8s
174:	learn: 3158.

308:	learn: 2890.1709780	total: 16.2s	remaining: 36.3s
309:	learn: 2890.0868068	total: 16.3s	remaining: 36.2s
310:	learn: 2889.7051366	total: 16.3s	remaining: 36.2s
311:	learn: 2889.3655560	total: 16.4s	remaining: 36.1s
312:	learn: 2888.5957387	total: 16.4s	remaining: 36s
313:	learn: 2886.4376552	total: 16.5s	remaining: 36s
314:	learn: 2886.3944829	total: 16.5s	remaining: 35.9s
315:	learn: 2886.3571281	total: 16.6s	remaining: 35.9s
316:	learn: 2886.1618663	total: 16.6s	remaining: 35.8s
317:	learn: 2885.8335164	total: 16.7s	remaining: 35.7s
318:	learn: 2884.6407746	total: 16.7s	remaining: 35.7s
319:	learn: 2884.6068530	total: 16.8s	remaining: 35.6s
320:	learn: 2884.5865153	total: 16.8s	remaining: 35.6s
321:	learn: 2884.0088529	total: 16.9s	remaining: 35.5s
322:	learn: 2883.7917577	total: 16.9s	remaining: 35.4s
323:	learn: 2883.7597064	total: 17s	remaining: 35.4s
324:	learn: 2883.2761741	total: 17s	remaining: 35.3s
325:	learn: 2883.0924091	total: 17.1s	remaining: 35.3s
326:	learn: 2882.6

463:	learn: 2839.9473173	total: 23.5s	remaining: 27.1s
464:	learn: 2839.8366786	total: 23.5s	remaining: 27.1s
465:	learn: 2839.8106279	total: 23.6s	remaining: 27s
466:	learn: 2839.7909573	total: 23.6s	remaining: 27s
467:	learn: 2839.7842090	total: 23.7s	remaining: 26.9s
468:	learn: 2839.7817278	total: 23.7s	remaining: 26.8s
469:	learn: 2839.7633488	total: 23.8s	remaining: 26.8s
470:	learn: 2839.6281401	total: 23.8s	remaining: 26.7s
471:	learn: 2839.6258877	total: 23.9s	remaining: 26.7s
472:	learn: 2839.6086455	total: 23.9s	remaining: 26.6s
473:	learn: 2839.6062797	total: 23.9s	remaining: 26.6s
474:	learn: 2839.6039968	total: 24s	remaining: 26.5s
475:	learn: 2839.6017936	total: 24s	remaining: 26.4s
476:	learn: 2839.5996671	total: 24s	remaining: 26.3s
477:	learn: 2839.5976142	total: 24s	remaining: 26.2s
478:	learn: 2839.5764200	total: 24.1s	remaining: 26.2s
479:	learn: 2839.5605034	total: 24.1s	remaining: 26.1s
480:	learn: 2839.5452203	total: 24.2s	remaining: 26.1s
481:	learn: 2839.52966

614:	learn: 2838.4190934	total: 30s	remaining: 18.8s
615:	learn: 2838.4189953	total: 30s	remaining: 18.7s
616:	learn: 2838.4168475	total: 30.1s	remaining: 18.7s
617:	learn: 2838.4163720	total: 30.1s	remaining: 18.6s
618:	learn: 2838.4143231	total: 30.1s	remaining: 18.6s
619:	learn: 2838.4094162	total: 30.2s	remaining: 18.5s
620:	learn: 2838.4093256	total: 30.2s	remaining: 18.4s
621:	learn: 2838.4028088	total: 30.3s	remaining: 18.4s
622:	learn: 2838.4027215	total: 30.3s	remaining: 18.3s
623:	learn: 2838.3979907	total: 30.3s	remaining: 18.3s
624:	learn: 2838.3908714	total: 30.4s	remaining: 18.2s
625:	learn: 2838.3907913	total: 30.4s	remaining: 18.2s
626:	learn: 2838.3903627	total: 30.4s	remaining: 18.1s
627:	learn: 2838.3899807	total: 30.5s	remaining: 18.1s
628:	learn: 2838.3893063	total: 30.5s	remaining: 18s
629:	learn: 2838.3892304	total: 30.5s	remaining: 17.9s
630:	learn: 2838.3887060	total: 30.6s	remaining: 17.9s
631:	learn: 2838.3872112	total: 30.6s	remaining: 17.8s
632:	learn: 2838

769:	learn: 2835.7065385	total: 36.1s	remaining: 10.8s
770:	learn: 2835.7063037	total: 36.1s	remaining: 10.7s
771:	learn: 2835.1882604	total: 36.2s	remaining: 10.7s
772:	learn: 2835.1224884	total: 36.2s	remaining: 10.6s
773:	learn: 2835.1206671	total: 36.2s	remaining: 10.6s
774:	learn: 2835.0500864	total: 36.3s	remaining: 10.5s
775:	learn: 2835.0483643	total: 36.3s	remaining: 10.5s
776:	learn: 2835.0467238	total: 36.3s	remaining: 10.4s
777:	learn: 2835.0364863	total: 36.3s	remaining: 10.4s
778:	learn: 2834.8911237	total: 36.4s	remaining: 10.3s
779:	learn: 2834.8908561	total: 36.4s	remaining: 10.3s
780:	learn: 2834.8908544	total: 36.4s	remaining: 10.2s
781:	learn: 2834.8678867	total: 36.5s	remaining: 10.2s
782:	learn: 2834.8677988	total: 36.5s	remaining: 10.1s
783:	learn: 2834.8677970	total: 36.5s	remaining: 10.1s
784:	learn: 2834.8592571	total: 36.6s	remaining: 10s
785:	learn: 2833.8836199	total: 36.6s	remaining: 9.96s
786:	learn: 2833.8822497	total: 36.6s	remaining: 9.91s
787:	learn: 

921:	learn: 2833.5044571	total: 42s	remaining: 3.55s
922:	learn: 2833.5040952	total: 42.1s	remaining: 3.51s
923:	learn: 2833.5040916	total: 42.1s	remaining: 3.46s
924:	learn: 2833.5007724	total: 42.1s	remaining: 3.42s
925:	learn: 2833.4976138	total: 42.2s	remaining: 3.37s
926:	learn: 2833.4973791	total: 42.3s	remaining: 3.33s
927:	learn: 2833.4970299	total: 42.3s	remaining: 3.28s
928:	learn: 2833.4966921	total: 42.3s	remaining: 3.24s
929:	learn: 2833.4936731	total: 42.4s	remaining: 3.19s
930:	learn: 2833.4907991	total: 42.4s	remaining: 3.14s
931:	learn: 2833.4907964	total: 42.5s	remaining: 3.1s
932:	learn: 2833.4904696	total: 42.5s	remaining: 3.05s
933:	learn: 2833.4904670	total: 42.5s	remaining: 3s
934:	learn: 2833.4904644	total: 42.5s	remaining: 2.96s
935:	learn: 2833.4901295	total: 42.6s	remaining: 2.91s
936:	learn: 2833.4873985	total: 42.6s	remaining: 2.87s
937:	learn: 2833.4847979	total: 42.7s	remaining: 2.82s
938:	learn: 2833.4845205	total: 42.7s	remaining: 2.77s
939:	learn: 2833

In [52]:
y_pred = ctbst.predict(X_test)
print(f'R2 metric on test data: {r2_score(y_test, y_pred):.6}')

R2 metric on test data: 0.90855


In [53]:
ctbst.save_model('model_final.cbm')